# 📘 Concept of Boosting — Expanded Notes & Demos

> This cell expands the slide **Concept of Boosting** with clear intuition, light math, and ready-to-run demo snippets (copy the code blocks into code cells).

---

## 1) What is Boosting?

Boosting builds a **strong learner** by adding many **weak learners** *sequentially*. Each new learner is trained to focus on the **mistakes** of the current ensemble.

A generic stage-wise additive model after \(T\) rounds:
\[
F_T(x)=\sum_{t=1}^{T}\alpha_t\,h_t(x)
\]
where \(h_t\) are weak learners (e.g., shallow trees) and \(\alpha_t\) their weights.

### AdaBoost (binary)
1) Initialize sample weights \(w_i^{(1)}=\tfrac{1}{N}\).  
2) Train weak learner \(h_t\) minimizing **weighted** error
\(\epsilon_t=\sum_i w_i^{(t)}\mathbf{1}[h_t(x_i)\neq y_i]\).  
3) Learner weight
\[
\alpha_t=\tfrac{1}{2}\ln\!\frac{1-\epsilon_t}{\epsilon_t}.
\]
4) Update sample weights (increase weight on misclassified points)
\[
w_i^{(t+1)}\propto w_i^{(t)}\exp\!\big(-\alpha_t y_i h_t(x_i)\big).
\]
5) Final classifier: \(\mathrm{sign}\!\left(\sum_t \alpha_t h_t(x)\right)\).

### Gradient Boosting (general loss)
Interpret boosting as **gradient descent in function space**. At round \(t\):
- Pseudo-residuals: \(r_i^{(t)}=-\left.\frac{\partial \ell(y_i,F(x_i))}{\partial F}\right|_{F=F_{t-1}(x_i)}\)
- Fit weak learner \(h_t\) to \(r^{(t)}\)
- Update \(F_t(x)=F_{t-1}(x)+\nu\,\alpha_t\,h_t(x)\) (learning rate \(\nu\in(0,1]\))

---

## 2) How is Boosting different from Bagging?

| Feature  | **Bagging** | **Boosting** |
|---|---|---|
| **Approach** | Models trained **in parallel** on bootstrapped samples | Models trained **sequentially**, each corrects prior errors |
| **Goal** | Mostly reduces **variance** via averaging | Mostly reduces **bias** by focusing on hard cases |
| **Base models** | Usually deeper trees (Random Forest) | Usually shallow trees (stumps) |
| **Examples** | Random Forest | AdaBoost, Gradient Boosting, XGBoost, LightGBM |

**Intuition:** Bagging = many opinions averaged. Boosting = iterative tutoring: *“what did we get wrong? let’s fix that.”*

---

## 3) Demo 1 — AdaBoost vs Bagging (Classification)

> Copy this block into a **code cell** to run.

```python
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt

def plot_decision_boundary(clf, X, y, title='Decision Boundary', h=0.05):
    x_min, x_max = X[:,0].min()-0.5, X[:,0].max()+0.5
    y_min, y_max = X[:,1].min()-0.5, X[:,1].max()+0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    plt.figure(figsize=(6,5))
    plt.contourf(xx, yy, Z, alpha=0.25)
    plt.scatter(X[:,0], X[:,1], c=y, edgecolor='k', s=25)
    plt.title(title); plt.xlabel('x1'); plt.ylabel('x2'); plt.show()

# Data
X, y = make_moons(n_samples=1200, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Models
stump = DecisionTreeClassifier(max_depth=1, random_state=42)
bag = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=3, random_state=42),
                        n_estimators=100, random_state=42)
ada = AdaBoostClassifier(base_estimator=stump, n_estimators=150, learning_rate=0.5, random_state=42)

bag.fit(X_train, y_train); ada.fit(X_train, y_train)
print("Bagging  accuracy:", accuracy_score(y_test, bag.predict(X_test)))
print("AdaBoost accuracy:", accuracy_score(y_test, ada.predict(X_test)))

plot_decision_boundary(bag, X_train, y_train, "Bagging (Randomized Trees) — Train")
plot_decision_boundary(ada, X_train, y_train, "AdaBoost (Decision Stumps) — Train")


# 📘 Gradient Boosting — Expanded Notes, Math & Demos

> **VS Code math:** enable `Markdown › Math: Enabled` (or use the *Markdown Preview Enhanced* extension) so formulas with `$...$` and `$$...$$` render properly.

---

## 1) What is Gradient Boosting?

**Gradient Boosting** is a boosting algorithm that builds a predictor **additively and sequentially** by minimizing a differentiable loss function via **gradient descent in function space**.

A stage-wise model after $T$ rounds:
$$
F_T(x)=\sum_{t=0}^{T} \nu\,\alpha_t\,h_t(x),\qquad 0<\nu\le1
$$
where $h_t$ are weak learners (typically shallow regression trees), $\alpha_t$ are step sizes (from line search or fixed), and $\nu$ is the **learning rate** (a.k.a. *shrinkage*).

---

## 2) How Gradient Boosting Works (Algorithm)

**Goal:** minimize the expected loss
$$
\min_F \; \mathbb{E}\big[\;\ell\!\left(y,\,F(x)\right)\;\big].
$$

**At iteration $t=1,2,\dots,T$:**

1. **Initialize model** (constant function):
   - For squared error: $F_0(x)=\arg\min_c \sum_i (y_i-c)^2=\bar y$.
   - For logistic loss: $F_0(x)=\arg\min_c \sum_i \log\!\big(1+e^{-y_i c}\big)$.

2. **Compute (pseudo)-residuals** (negative gradients):
   $$
   r_i^{(t)} \;=\; -\left.\frac{\partial \ell\!\left(y_i, F(x_i)\right)}{\partial F}\right|_{F=F_{t-1}(x_i)} .
   $$

   - **Squared error** $\ell(y,F)=\tfrac12(y-F)^2 \Rightarrow r_i^{(t)} = y_i - F_{t-1}(x_i)$ (ordinary residuals).
   - **Logistic (binary, $y\in\{-1,+1\}$)** $\ell(y,F)=\log(1+e^{-yF}) \Rightarrow r_i^{(t)} = \frac{y_i}{1+e^{y_i F_{t-1}(x_i)}}$.

3. **Fit a weak learner** to residuals:
   $$
   h_t \;\approx\; \arg\min_h \sum_i \big(r_i^{(t)} - h(x_i)\big)^2 .
   $$

4. **Line search (optional)** for step size:
   $$
   \alpha_t \;=\; \arg\min_\alpha \sum_i \ell\!\left(y_i,\; F_{t-1}(x_i)+\alpha\,h_t(x_i)\right).
   $$

5. **Update the model** with learning rate $\nu$:
   $$
   F_t(x) \;=\; F_{t-1}(x) + \nu\,\alpha_t\,h_t(x).
   $$

6. **Repeat** until $t=T$ or an early-stopping criterion is met.

---

## 3) Why it Works

- **Bias reduction:** each $h_t$ targets the remaining structure (residuals) the current ensemble misses.  
- **Functional gradient descent:** steps follow the steepest descent direction of the empirical loss in the space of functions.  
- **Trees as base learners:** piecewise-constant learners approximate complex functions via many small additive corrections.

---

## 4) Regularization & Practical Tips

- **Learning rate ($\nu$):** smaller $\nu$ $\Rightarrow$ slower learning but better generalization; increase $T$ accordingly.  
- **Tree complexity:** control with `max_depth`, `min_samples_leaf`, `max_leaf_nodes`.  
- **Subsampling:** use a fraction $m$ of data each iteration (*stochastic gradient boosting*), which adds regularization.  
- **Early stopping:** monitor validation loss; stop when it plateaus.  
- **Feature interaction depth:** tree depth $\approx$ max interaction order learned.

---

## 5) Demo — Regression (Squared Error)

> Copy into a **code cell** to run.

```python
import numpy as np, matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Synthetic data
rng = np.random.RandomState(0)
X = np.sort(5*rng.rand(800,1), axis=0)
y_true = np.sin(X).ravel()
y = y_true + 0.25*rng.randn(X.shape[0])

Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=42)

gbr = GradientBoostingRegressor(
    n_estimators=500, learning_rate=0.05,
    max_depth=2, subsample=0.8, random_state=0
)
rf = RandomForestRegressor(n_estimators=300, random_state=0)
dt = DecisionTreeRegressor(max_depth=3, random_state=0)

for m in (gbr, rf, dt):
    m.fit(Xtr, ytr)

xs = np.linspace(0,5,500).reshape(-1,1)
plt.figure(figsize=(7,4))
plt.scatter(Xtr, ytr, s=10, alpha=0.4, label="train")
plt.plot(xs, np.sin(xs).ravel(), lw=2, label="true sin(x)")
plt.plot(xs, gbr.predict(xs), lw=2, label="GBR")
plt.plot(xs, rf.predict(xs),  lw=2, label="RandomForest")
plt.plot(xs, dt.predict(xs),  lw=2, label="DecisionTree")
plt.legend(); plt.xlabel("x"); plt.ylabel("y"); plt.title("Gradient Boosting (Regression)")
plt.show()

print("MSE (test) — GBR:", mean_squared_error(yte, gbr.predict(Xte)))
print("MSE (test) — RF :", mean_squared_error(yte, rf.predict(Xte)))
print("MSE (test) — DT :", mean_squared_error(yte, dt.predict(Xte)))


# 📘 Gradient Boosting — Key Parameters, Math, and Tuning Playbook

> **VS Code math:** enable `Markdown › Math: Enabled` (or use *Markdown Preview Enhanced*) so `$...$` and `$$...$$` render.

---

## 1) Learning Rate ($\nu$ — “shrinkage”)

- **Role:** scales each stage’s contribution in the additive model  
  $$
  F_t(x) \;=\; F_{t-1}(x) \;+\; \nu\,\alpha_t\,h_t(x),\qquad 0<\nu\le1
  $$
- **Effect:** smaller $\nu$ $\Rightarrow$ **more conservative** steps, typically **better generalization** if you increase the number of stages $T$.
- **Rule of thumb:** treat the **effective capacity** roughly like $T\!\times\!\nu$ (not exact, but helpful). To keep capacity similar, halving $\nu$ often means doubling $T$.
- **Typical ranges:**  
  - tabular data: $\nu \in [0.01, 0.1]$ for large datasets; $\nu \in [0.05, 0.3]$ for smaller datasets.  
  - combine with **early stopping** on a validation set.

---

## 2) Number of Estimators ($T$ — stages/trees)

- **Role:** how many weak learners you add **sequentially**.
- **Bias–variance:** increasing $T$ reduces **bias** at first; after some point it may increase **variance** (overfitting) unless regularized (small $\nu$, shallow trees, subsampling).
- **Early stopping:** monitor validation loss and stop when it fails to improve for $p$ rounds.

> **Heuristic pairing:** search over $(\nu, T)$ pairs with a fixed **interaction depth** (tree `max_depth`):
> - coarse grid: $\nu \in \{0.3, 0.1, 0.05, 0.02\}$ and $T \in \{100, 300, 600, 1200, 2000\}$  
> - enable early stopping to avoid wasting rounds.

---

## 3) Regularization (what keeps boosting from overfitting)

### (a) Tree complexity
- Limit **interaction order** via `max_depth` (or `max_leaf_nodes`).  
  Shallow trees (depth 1–3) are common: they learn low-order interactions.
- Enforce **leaf size** (`min_samples_leaf`) to smooth predictions.

### (b) Subsampling / Stochastic GB
- Use a fraction $m$ of data each round ($m\in[0.5,1)$):
  $$\text{fit } h_t \text{ on a random subset } \mathcal{S}_t,\ |\mathcal{S}_t|=mN.$$
- Adds noise to the gradient step $\Rightarrow$ acts like **regularization** and often speeds up training.

### (c) Column subsampling (if available)
- Sample a subset of features per split or per tree. Helps decorrelate learners and reduce variance.

### (d) Penalizing leaf values (L2)
- For a terminal region $R_{tj}$ with residuals $\{r_i^{(t)}: x_i\in R_{tj}\}$, choose the leaf value
  $$
  \gamma_{tj} \;=\; \arg\min_{\gamma}\ \sum_{x_i\in R_{tj}} \big(r_i^{(t)}-\gamma\big)^2 \;+\; \lambda\,\gamma^2
  \;\;\Rightarrow\;\;
  \gamma_{tj} \;=\; \frac{\sum r_i^{(t)}}{|R_{tj}|+\lambda}.
  $$
- Many libraries expose this via a **lambda**/`reg_lambda` parameter (XGBoost/LightGBM).

### (e) Learning-rate shrinkage (again!)
- The single most robust regularizer: **smaller $\nu$** with **larger $T$**.

---

## 4) Putting It Together — Tuning Recipe

1. **Fix depth** (interaction order), e.g. `max_depth = 2` (classification) or `2–4` (regression).
2. **Search $(\nu, T)$** with early stopping (patience $p\approx 25$ rounds).
3. Add **subsampling**: `subsample = 0.6–0.9`; optionally `colsample` if supported.
4. Adjust **leaf size** (`min_samples_leaf`) to smooth predictions.
5. If available, enable **L2 on leaves** (small `lambda`: $10^{-3}$–$10^{-1}$).
6. Revisit **depth** to capture needed interactions without overfitting.

---

## 5) Minimal Code Patterns (copy into code cells)

### Early stopping grid for $(\nu, T)$ (scikit-learn style)
```python
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import log_loss
import numpy as np

X, y = make_classification(n_samples=20000, n_features=40, n_informative=15,
                           class_sep=1.0, random_state=0)
Xtr, Xva, ytr, yva = train_test_split(X, y, test_size=0.2, random_state=42)

candidates = [(0.3, 300), (0.1, 600), (0.05, 1200), (0.02, 2000)]
best = None

for lr, n in candidates:
    clf = GradientBoostingClassifier(
        learning_rate=lr, n_estimators=n, max_depth=2,
        subsample=0.8, validation_fraction=0.2, n_iter_no_change=25,
        random_state=0
    ).fit(Xtr, ytr)
    p = clf.predict_proba(Xva)
    score = log_loss(yva, p)
    print((lr, n), "val logloss:", score)
    best = (score, (lr, n), clf) if best is None or score < best[0] else best

print("Best:", best[1], "val logloss:", best[0])


In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

data = load_breast_cancer()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


print(f"Features: {data.feature_names}")
print(f"Classes: {data.target_names}")

Features: ['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
Classes: ['malignant' 'benign']


In [3]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report

gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train, y_train)

y_pred_gb = gb_model.predict(X_test)
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print(f"Gradient Boosting Accuracy: {accuracy_gb}")

Gradient Boosting Accuracy: 0.956140350877193


In [4]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7]
}

grid_search = GridSearchCV(
    estimator=GradientBoostingClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs = -1
)

grid_search.fit(X_train, y_train)




,estimator,GradientBoost...ndom_state=42)
,param_grid,"{'learning_rate': [0.01, 0.1, ...], 'max_depth': [3, 5, ...], 'n_estimators': [50, 100, ...]}"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,loss,'log_loss'


In [5]:
print(f"Best Parameters: {grid_search.best_params_}")

Best Parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}


In [6]:
print(f"Best Cross-Validations Params: {grid_search.best_score_}")

Best Cross-Validations Params: 0.9648351648351647


In [7]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy: {accuracy_rf}")

Random Forest Accuracy: 0.9649122807017544
